# **1. Perkenalan Dataset**


Tahap pertama, Anda harus mencari dan menggunakan dataset dengan ketentuan sebagai berikut:

1. **Sumber Dataset**:  
   https://www.kaggle.com/datasets/shivamb/machine-predictive-maintenance-classification 

2. **Gambaran Umum**:  
   Dataset ini merupakan dataset Predictive Maintenance (Pemeliharaan Prediktif) yang berisi data sensor dan informasi operasional dari mesin industri. Dataset ini dirancang untuk memprediksi kegagalan mesin dan mengidentifikasi jenis kegagalan yang mungkin terjadi. 
3. **Karakteristik Dataset**:

   Jumlah Data
      50.000 baris data pengamatan
      10 kolom variabel/fitur

   Variabel dalam Dataset
      1. UDI (Unique Device Identifier): ID unik untuk setiap pengamatan (1-50000)

      2. Product ID: Identifikasi produk dengan format kombinasi huruf (L/M/H) dan angka
         - L: Low quality (Kualitas rendah)
         - M: Medium quality (Kualitas sedang)
         - H: High quality (Kualitas tinggi)
      
      3. Type: Kategori kualitas produk (L, M, atau H)

      4. Air temperature [K]: Suhu udara dalam Kelvin (berkisar 295-305K)

      5. Process temperature [K]: Suhu proses dalam Kelvin (berkisar 305-314K)

      6. Rotational speed [rpm]: Kecepatan rotasi mesin dalam RPM (1168-2886 rpm)

      7. Torque [Nm]: Torsi mesin dalam Newton-meter (3.8-76.6 Nm)

      8. Tool wear [min]: Tingkat keausan alat dalam menit (0-253 menit)

      9. Target: Variabel target biner (0: tidak gagal, 1: gagal)

      10. Failure Type: Jenis kegagalan spesifik, termasuk:
            - No Failure (Tidak ada kegagalan)
            - Heat Dissipation Failure (Kegagalan disipasi panas)
            - Power Failure (Kegagalan daya)
            - Overstrain Failure (Kegagalan karena beban berlebih)
            - Tool Wear Failure (Kegagalan karena keausan alat)
            - Random Failures (Kegagalan acak)



# **2. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning atau deep learning.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import pandas as pd

# **3. Memuat Dataset**

Pada tahap ini, Anda perlu memuat dataset ke dalam notebook. Jika dataset dalam format CSV, Anda bisa menggunakan pustaka pandas untuk membacanya. Pastikan untuk mengecek beberapa baris awal dataset untuk memahami strukturnya dan memastikan data telah dimuat dengan benar.

Jika dataset berada di Google Drive, pastikan Anda menghubungkan Google Drive ke Colab terlebih dahulu. Setelah dataset berhasil dimuat, langkah berikutnya adalah memeriksa kesesuaian data dan siap untuk dianalisis lebih lanjut.

Jika dataset berupa unstructured data, silakan sesuaikan dengan format seperti kelas Machine Learning Pengembangan atau Machine Learning Terapan

In [ ]:
df = pd.read_csv('D:\Eksperimen_SML_Zayga\predictive_maintenance.raw')
df.head()

# **4. Exploratory Data Analysis (EDA)**

Pada tahap ini, Anda akan melakukan **Exploratory Data Analysis (EDA)** untuk memahami karakteristik dataset.

Tujuan dari EDA adalah untuk memperoleh wawasan awal yang mendalam mengenai data dan menentukan langkah selanjutnya dalam analisis atau pemodelan.

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.duplicated().sum()

In [ ]:
df.isnull().sum()

In [ ]:
# Melihat korelasi antar fitur
corr_matrix = df.corr(numeric_only=True)

# Visualisasi korelasi dengan heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Heatmap Korelasi Antar Fitur')
plt.show()

In [ ]:
df.hist(figsize=(12, 8), bins=30)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(15, 10))
df.select_dtypes(include=np.number).boxplot()
plt.title('Boxplot untuk Deteksi Outlier pada Fitur Numerik')
plt.xticks(rotation=45)
plt.show()

# **5. Data Preprocessing**

Pada tahap ini, data preprocessing adalah langkah penting untuk memastikan kualitas data sebelum digunakan dalam model machine learning.

Jika Anda menggunakan data teks, data mentah sering kali mengandung nilai kosong, duplikasi, atau rentang nilai yang tidak konsisten, yang dapat memengaruhi kinerja model. Oleh karena itu, proses ini bertujuan untuk membersihkan dan mempersiapkan data agar analisis berjalan optimal.

Berikut adalah tahapan-tahapan yang bisa dilakukan, tetapi **tidak terbatas** pada:
1. Menghapus atau Menangani Data Kosong (Missing Values)
2. Menghapus Data Duplikat
3. Normalisasi atau Standarisasi Fitur
4. Deteksi dan Penanganan Outlier
5. Encoding Data Kategorikal
6. Binning (Pengelompokan Data)

Cukup sesuaikan dengan karakteristik data yang kamu gunakan yah. Khususnya ketika kami menggunakan data tidak terstruktur.

## **5.1. Cek Kolom Kategorikal**

In [ ]:
# Cek kolom kategorikal
categorical_cols = df.select_dtypes(include=['object']).columns
print("Kolom kategorikal:", categorical_cols.tolist())
print("\nDistribusi nilai pada kolom kategorikal:")
for col in categorical_cols:
    print(f"\n{col}:")
    print(df[col].value_counts())

## **5.2. Encoding Data Kategorikal**

In [ ]:
from sklearn.preprocessing import LabelEncoder
# Membuat copy dataframe untuk preprocessing
df_processed = df.copy()

# Encoding kolom Type menggunakan Label Encoding
if 'Type' in df_processed.columns:
    le = LabelEncoder()
    df_processed['Type'] = le.fit_transform(df_processed['Type'])
    print("Encoding Type:")
    print(f"Mapping: {dict(zip(le.classes_, le.transform(le.classes_)))}")

# Hapus kolom yang tidak diperlukan (jika ada kolom ID atau Product ID)
columns_to_drop = ['UDI', 'Product ID'] if 'Product ID' in df_processed.columns else ['UDI']
if all(col in df_processed.columns for col in columns_to_drop):
    df_processed = df_processed.drop(columns=columns_to_drop)
    print(f"\nKolom yang dihapus: {columns_to_drop}")

print("\nDataset setelah encoding:")
df_processed.head()

## **5.3. Handling Outlier (Metode IQR)**

In [ ]:
# Fungsi untuk mendeteksi dan menangani outlier menggunakan IQR
def handle_outliers_iqr(df, columns, method='cap'):
    """
    Menangani outlier menggunakan metode IQR
    method: 'cap' untuk capping, 'remove' untuk menghapus outlier
    """
    df_clean = df.copy()
    outlier_info = {}
    
    for col in columns:
        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Hitung jumlah outlier
        outliers = df_clean[(df_clean[col] < lower_bound) | (df_clean[col] > upper_bound)]
        outlier_info[col] = len(outliers)
        
        if method == 'cap':
            # Capping: ganti outlier dengan batas atas/bawah
            df_clean[col] = df_clean[col].clip(lower=lower_bound, upper=upper_bound)
        elif method == 'remove':
            # Hapus baris yang mengandung outlier
            df_clean = df_clean[(df_clean[col] >= lower_bound) & (df_clean[col] <= upper_bound)]
    
    return df_clean, outlier_info

# Pilih kolom numerik untuk handling outlier (kecuali Target)
numeric_cols = df_processed.select_dtypes(include=[np.number]).columns.tolist()
if 'Target' in numeric_cols:
    numeric_cols.remove('Target')

print(f"Jumlah data sebelum handling outlier: {len(df_processed)}")

# Handling outlier dengan metode capping
df_processed, outlier_counts = handle_outliers_iqr(df_processed, numeric_cols, method='cap')

print(f"Jumlah data setelah handling outlier: {len(df_processed)}")
print("\nJumlah outlier per kolom:")
for col, count in outlier_counts.items():
    print(f"{col}: {count} outliers")

## **5.4. Standarisasi Fitur Numerik**

In [ ]:
# Pisahkan fitur (X) dan target (y)
X = df_processed.drop('Target', axis=1)
y = df_processed['Target']

print(f"Shape fitur (X): {X.shape}")
print(f"Shape target (y): {y.shape}")
print(f"\nDistribusi kelas target:")
print(y.value_counts())
print(f"\nPersentase kelas:")
print(y.value_counts(normalize=True) * 100)

In [ ]:
# Simpan hasil preprocessing ke file CSV
df_processed.to_csv('D:\\Eksperimen_SML_Zayga\\preprocessing\\predictive_maintenance_processed.csv', index=False)
print("Hasil preprocessing berhasil disimpan ke predictive_maintenance_processed.csv")